# Parent Document Retrieval
Hệ thống retrieval sử dụng child chunks để tìm kiếm và trả về parent chunks với đầy đủ context

In [1]:
# Tạo đường dẫn chung để đọc utils
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
# Import các thư viện cần thiết
import json
from pinecone.grpc import PineconeGRPC as Pinecone
from dotenv import load_dotenv
import os
from sentence_transformers import SentenceTransformer
import numpy as np
from utils.load_chunks_json import load_chunks_from_json

In [3]:
# Khởi tạo các biến môi trường
load_dotenv("../.env")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HOST_DENSE = os.getenv("HOST_DENSE")

# Khởi tạo embedding model
embedding_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")
embedding_model.max_seq_length = 2048

print(f"Embedding model dimension: {embedding_model.get_sentence_embedding_dimension()}")

Embedding model dimension: 1024


In [4]:
# Kết nối Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
dense_index = pc.Index(host=HOST_DENSE)

print("Đã kết nối thành công với Pinecone")

Đã kết nối thành công với Pinecone


In [35]:
def parent_document_search(query, namespace="lich-su-dang-children", top_k=10, alpha=0.7):
    """
    Thực hiện Parent Document Retrieval
    
    Args:
        query: Câu hỏi tìm kiếm của người dùng
        namespace: Namespace chứa child chunks trong Pinecone
        top_k: Số lượng parent chunks trả về
        alpha: Trọng số cho việc ranking (0-1)
               alpha = 1: chỉ dựa vào similarity score
               alpha = 0: chỉ dựa vào số lượng child chunks
    
    Returns:
        List các parent chunks được rank theo độ relevance
    """
    
    # Bước 1: Embed query thành vector
    print(f"Đang embed query: '{query}'")
    query_vector = embedding_model.encode(query).tolist()
    
    # Bước 2: Tìm kiếm child chunks có score cao
    # Lấy nhiều child chunks để có đủ parent chunks đa dạng
    child_results = dense_index.query(
        vector=query_vector,
        top_k=top_k * 2,  # Lấy gấp 3 lần để đảm bảo có đủ parent unique
        include_metadata=True,
        namespace=namespace
    )
    
    print(f"Tìm được {len(child_results.matches)} child chunks")
    
    # Bước 3: Gom nhóm child chunks theo parent_id
    parent_scores = {}  # parent_id -> tổng score
    parent_child_count = {}  # parent_id -> số lượng child chunks
    parent_best_score = {}  # parent_id -> score cao nhất
    
    for match in child_results.matches:
        parent_id = match.metadata.get('parent_id')
        
        if parent_id:
            # Khởi tạo nếu chưa có
            if parent_id not in parent_scores:
                parent_scores[parent_id] = 0
                parent_child_count[parent_id] = 0
                parent_best_score[parent_id] = 0
            
            # Cộng dồn score và đếm child chunks
            parent_scores[parent_id] += match.score
            parent_child_count[parent_id] += 1
            parent_best_score[parent_id] = max(parent_best_score[parent_id], match.score)
    
    print(f"Gom được {len(parent_scores)} parent IDs unique")
    
    # Bước 4: Tính score tổng hợp và rank các parent chunks
    ranked_parents = []
    
    for parent_id, total_score in parent_scores.items():
        # Kiểm tra parent_id có tồn tại trong lookup table không
        avg_score = total_score / parent_child_count[parent_id]
        child_count = parent_child_count[parent_id]
        best_score = parent_best_score[parent_id]
        
        # Normalize child count (giả sử max reasonable là 10 child chunks)
        normalized_child_count = min(child_count / 10.0, 1.0)
        
        # Score cuối = alpha * avg_similarity + (1-alpha) * normalized_child_count
        final_score = alpha * avg_score + (1 - alpha) * normalized_child_count
        
        # Lấy parent chunk từ lookup table
        parent_chunk = dense_index.fetch(
            ids=[parent_id],
            namespace="lich-su-dang"
        )
        
        parent_chunk = dense_index.fetch(
            ids=[f"{parent_id}"],
            namespace="lich-su-dang"
        )['vectors'][f"{parent_id}"]
                
        # Thêm vào kết quả
        ranked_parents.append({
            'parent_chunk': parent_chunk,
            'parent_id': parent_id,
            'score': final_score,
            'avg_child_score': avg_score,
            'best_child_score': best_score,
            'total_child_score': total_score,
            'child_count': child_count
        })
    
    # Bước 5: Sắp xếp theo score giảm dần và trả về top_k
    ranked_parents.sort(key=lambda x: x['score'], reverse=True)
    
    print(f"Ranked {len(ranked_parents)} parent chunks, trả về top {top_k}")
    
    return ranked_parents[:top_k]

In [57]:
test_query = "Trình bày những biện pháp nhân nhượng của quân ta đối với quân Tưởng"

print(f"Query: {test_query}")
print("=" * 60)

results = parent_document_search(test_query)

print(f"\nKết quả tìm kiếm: {len(results)} parent chunks")
print("=" * 60)

Query: Trình bày những biện pháp nhân nhượng của quân ta đối với quân Tưởng
Đang embed query: 'Trình bày những biện pháp nhân nhượng của quân ta đối với quân Tưởng'
Tìm được 20 child chunks
Gom được 13 parent IDs unique
Ranked 13 parent chunks, trả về top 10

Kết quả tìm kiếm: 10 parent chunks


In [62]:
from google import genai

def generate_answer(input_query, context):
    prompt = f"""
    Mày là một chuyên gia trong việc trả lời các môn học đại cương về chính trị bậc đại học không chính quy.
    
    Dưới đây chính là query của người dùng về các câu hỏi liên quan tới các môn chính trị. 
    {input_query}
    
    Còn dưới đây là những context được cung cấp đên mày để trả lời câu hỏi của người dùng.
    {context}
    
    Yêu cầu khi trả lời:
    - Tóm tắt và tổng hợp thông tin từ các đoạn context có liên quan.
    - Hạn chế sao chép nguyên văn toàn bộ một đoạn nào từ context.
    - Văn phong rõ ràng, súc tích, mang tính học thuật.
    - Hãy ghi nguồn gốc của thông tin trong câu trả lời bằng id của đoạn văn bản trong context, đặt id ở cuối thông tin đó.
    - Dựa vào nhưng thông tin trên, mày hãy thực hiện trả lời câu hỏi của người dùng. Chỉ trả lời theo nội dung context cung cấp. Nếu những nội dung đó không liên quan đến câu hỏi thì trả lời "Context không liên quan".
    """
    # prompt = f"""Bạn là một Generator chuyên nghiệp, được giao nhiệm vụ trả lời câu hỏi dựa trên các đoạn văn bản sau (context). 
    # ---

    # Câu hỏi: {input_query}

    # Context:
    # {context}
    # Context là một tuple (metadata, id) sẽ chứa metadata, trong metadata sẽ có content là nội dung để trả lời câu hỏi. Metadata là thông tin bổ sung cho trích dẫn nguồn.
    # Phần id cũng sử dụng để trích dẫn nguồn. Thêm vào để xác định nguồn gốc của thông tin trong câu trả lời. Hãy thêm vào cuối câu trả lời liên quan. 
    
    # QUAN TRỌNG: 
    # - *Chỉ được sử dụng thông tin có trong context.* 
    # - *Không được đưa ra bất kỳ suy đoán, bổ sung, hay kiến thức bên ngoài nào.* 
    # - Nếu không tìm thấy câu trả lời trong context, hãy trả lời: *"Thông tin không có trong đoạn văn được cung cấp."*

    # Yêu cầu khi trả lời:
    # - Tóm tắt và tổng hợp thông tin từ các đoạn context có liên quan.
    # - Hạn chế sao chép nguyên văn toàn bộ một đoạn nào từ context.
    # - Văn phong rõ ràng, súc tích, mang tính học thuật.
    # - Hãy ghi nguồn gốc của thông tin trong câu trả lời bằng id của đoạn văn bản trong context, đặt id ở cuối thông tin đó.
    # """
    
    
    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
    )

    return response.text

In [ ]:
results[0]['parent_id']

'LSD_chunk_00093'

In [63]:
context_list = [(result['parent_chunk']['metadata'], result['parent_id']) for result in results]
len(context_list)
result = generate_answer(test_query, context_list)
result

'Để đối phó với âm mưu "diệt Cộng, cầm Hồ, phá Việt Minh" của quân Tưởng Giới Thạch và tay sai, Đảng và Chính phủ Hồ Chí Minh đã chủ trương thực hiện sách lược "hòa hoãn, nhân nhượng có nguyên tắc" nhằm bảo vệ chính quyền cách mạng non trẻ. Các biện pháp nhân nhượng cụ thể bao gồm:\n\n1.  **Về chính trị và tổ chức:**\n    *   Đảng Cộng sản Đông Dương tự nguyện công bố "tự giải tán" vào ngày 11-11-1945, rút vào hoạt động bí mật và chỉ để lại một bộ phận hoạt động công khai dưới danh nghĩa "Hội nghiên cứu chủ nghĩa Mác ở Đông Dương" để tránh mũi nhọn tấn công của kẻ thù. (LSD_chunk_00102)\n    *   Sau cuộc bầu cử Quốc hội, Chủ tịch Hồ Chí Minh đã chấp nhận mở rộng thành phần đại biểu Quốc hội bằng cách bổ sung thêm 70 ghế không qua bầu cử cho một số đảng viên thuộc Việt Cách, Việt Quốc – các tổ chức tay sai của Tưởng. (LSD_chunk_00102)\n    *   Chính phủ liên hiệp được cải tổ và mở rộng, với sự tham gia của nhiều nhân sĩ, trí thức, người không đảng phái, và cả một số phần tử cầm đầu các 

In [64]:
print(result)

Để đối phó với âm mưu "diệt Cộng, cầm Hồ, phá Việt Minh" của quân Tưởng Giới Thạch và tay sai, Đảng và Chính phủ Hồ Chí Minh đã chủ trương thực hiện sách lược "hòa hoãn, nhân nhượng có nguyên tắc" nhằm bảo vệ chính quyền cách mạng non trẻ. Các biện pháp nhân nhượng cụ thể bao gồm:

1.  **Về chính trị và tổ chức:**
    *   Đảng Cộng sản Đông Dương tự nguyện công bố "tự giải tán" vào ngày 11-11-1945, rút vào hoạt động bí mật và chỉ để lại một bộ phận hoạt động công khai dưới danh nghĩa "Hội nghiên cứu chủ nghĩa Mác ở Đông Dương" để tránh mũi nhọn tấn công của kẻ thù. (LSD_chunk_00102)
    *   Sau cuộc bầu cử Quốc hội, Chủ tịch Hồ Chí Minh đã chấp nhận mở rộng thành phần đại biểu Quốc hội bằng cách bổ sung thêm 70 ghế không qua bầu cử cho một số đảng viên thuộc Việt Cách, Việt Quốc – các tổ chức tay sai của Tưởng. (LSD_chunk_00102)
    *   Chính phủ liên hiệp được cải tổ và mở rộng, với sự tham gia của nhiều nhân sĩ, trí thức, người không đảng phái, và cả một số phần tử cầm đầu các tổ chứ